In [ ]:
# ok what is the simplest thing I can start with? Just a multi armed bandit...

# this should just look like a very simple script....

# What is my goal? Have a clear idea and you may get what you need.
# Utlimate goal is to get up robot arms, have them just attempt to grasp objects, and then come back 1 week later, and their success rate is near 100%...

# Idea is to build up from simple to complex grasps and motion...


# What does good look like? Maximise reward, maximise success rate, fast computation, good deployment performance

In [ ]:
# policy_config = dict()
# env_config = dict()
# policy = Policy.make(policy_config) # For now lets just directly construct it
# env = Env.make(env_config) # I can just use the gym registration system

Dimensions of the RL problem

- Action space size (larger is harder)
- State space size (large is harder)
- Episode Time horizion (further away is harder, credit assignment problem)
- History/Context Req./Markov property (More context is harder, essentially greater state space, blind spots in driving, you cannot forget about the car)
- World Dynamics -> are they constatly changing? Is it easy to predict? Are there other agents?


How to build up?

We start with very simple gym and envs ex1_classic_bandit -> at some point we get to ex20 type thing :)

How is this different than the full robotics problem?

1. Here we assume we know the full state, in reality we sensors are noisey, kinematic chains are not accurate, etc.
2. We are not worried about execution* that is an offline dataset or simulator

action = policy(state)

state, reward, terminated, truncated, info = env.step(action)

Here was my idea to deal with the delayed rewards! don't return until you have the reward, so you can wait inside the environment, everything else is outside your control, so this make sense..



In [ ]:
%load_ext autoreload
%autoreload 2

I am getting import errors for some bam gym py types etc...
That is not good! this really should just be in numpy land, etc... clear away all of these distractions please.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import bam_gym # import to that __init__.py is run to register envs
from bam_gym import print_reset, print_step

In [ ]:

# Square numbers are easy to see as square
n_arms = 4**2
env = gym.make("bam_local/ClassicBandit", render_mode="human", clip=10, n_arms=n_arms)
print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)
print("Reset:", env.reset())

env.render()

In [ ]:
state, info = env.reset()
print_reset(state, info)

def reset_env(env, terminated, truncated, info) -> bool:
    if terminated or truncated:
        if env.metadata['autoreset_mode'] != gym.vector.AutoresetMode.SAME_STEP:
            return True

    return False

for i in range(10):
    action = env.action_space.sample()
    # action = policy(state)
    state, reward, terminated, truncated, info = env.step(action)
    print_step(action, state, reward, terminated, truncated, info, i)

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()

env.close()

In [ ]:
# I can use a super duper simple policy to beat this, essentially just average the rewards to date, and then pick the highest action with epislon greedy probability
# If you want to get a very accuarte understanding of the world, then you can continue to do random exploration, but to actually maximise reward you want to exploit as well!

class AveragePolicy():
    def __init__(self, n_actions, epsilon=0.2):
        self.n_actions = n_actions
        self.epsilon = epsilon
        self.iter = 0
        self.attempts = np.ones(n_actions) # avoid divide by zero
        self.rewards = np.ones(n_actions)/n_actions
        self.W = self.rewards / self.attempts
        # self.W = np.ones(n_actions)/n_actions


    def __call__(self, state):
        self.iter += 1
        if np.random.random() <= self.epsilon:
            return np.random.randint(self.n_actions)
        else:
            return np.argmax(self.W)

    def update(self, action, reward):
        self.attempts[action] += 1
        self.rewards[action] += reward
        self.W = self.rewards / self.attempts


# Be a little fancy and calculing in a moving average way :)
#     def update(self, action, reward):
#         self.attempts[action] += 1
#         self.rewards[action] += reward
#         self.W = self.W * (1 - self.alpha) + self.rewards / self.attempts * self.alpha


policy = AveragePolicy(n_actions=n_arms, epsilon=0.2)

In [ ]:
reward_history = []

In [ ]:
state, info = env.reset()
# policy.epsilon = 0.0 # Manually tune epsilon 

for i in range(1000):
    action = policy(state)
    state, reward, terminated, truncated, info = env.step(action)
    # print_step(action, state, reward, terminated, truncated, info, i)
    policy.update(action, reward)
    reward_history.append(reward)

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()

env.close()

In [ ]:
print(policy.attempts)
print(policy.rewards)

width = 0.4
x = np.arange(n_arms)

plt.figure(figsize=(10, 6))


plt.bar(x - width/2, env.unwrapped.arm_thresholds, width, alpha=0.7, label="Actual")
plt.bar(x + width/2, policy.W, width, alpha=0.7, label="Estimated")

plt.ylim(0, 1)

plt.xlabel('Arm')
plt.ylabel('Threshold')
plt.title('Classic Bandit Arms: Thresholds')
plt.legend()

plt.grid(True)
plt.show()

In [ ]:
plt.plot(reward_history, label="Reward")

# Compute moving average
window_size = 50
if len(reward_history) >= window_size:
    moving_avg = np.convolve(reward_history, np.ones(window_size)/window_size, mode='valid')
    plt.plot(range(window_size-1, len(reward_history)), moving_avg, label=f"Moving Avg ({window_size})", color='orange')

plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Reward History and Moving Average")
plt.legend()
plt.show()

In [ ]:

# One idea is to view as image... better I think to compare agianst the ground turth bar chart though!

W = policy.W  # assume it's a 1D numpy array
n = W.shape[0]

# Find next perfect square
square_dim = int(np.ceil(np.sqrt(n)))
padded_len = square_dim ** 2

# Pad with zeros (or another value if you want, e.g. np.nan)
W_padded = np.pad(W, (0, padded_len - n), mode="constant", constant_values=np.nan)

# Reshape to square
W_square = W_padded.reshape(square_dim, square_dim)
print(W.shape)
fig, ax = plt.subplots()
im = ax.imshow(W_square, cmap="viridis", interpolation="nearest")

# Show ticks at integer positions (pixel centers)
ax.set_xticks(np.arange(W_square.shape[1]))
ax.set_yticks(np.arange(W_square.shape[0]))

# Add gridlines between pixels if you like
ax.set_xticks(np.arange(-0.5, W_square.shape[1], 1), minor=True)
ax.set_yticks(np.arange(-0.5, W_square.shape[0], 1), minor=True)

plt.colorbar(im)
plt.show()
